# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [ ]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1

## Experiments with segmentation methods

In [ ]:
import os
import datasets
import datetime
import re
import pandas as pd

from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('api_key')

### MedQA-USMLE-4-options dataset

In [ ]:
# run experiments on train split
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
data = data.add_column('original_quest_id', range(len(data)))

In [ ]:
# sample questions for experiments
sampled_data = data.shuffle(seed=42).select(range(50))

### MedQA dataset

In [ ]:
# download MedQA data from https://drive.google.com/file/d/1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw/view?usp=sharing
!gdown -q 1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw
!unzip -q data_clean.zip -d /content/medQA

### Document embedding and indexing

In [ ]:
def load_documents():
    '''
    Loads reference textbooks for MedQA as a list of Document objects.
    '''

    medqa_path = '/content/medQA/data_clean/textbooks/en/'
    loader = DirectoryLoader(medqa_path, glob='**/*.txt', loader_cls=TextLoader)

    # list of 'Document' files, with 'source'/file path as metadata
    documents = loader.load()

    return documents

In [ ]:
def create_vector_store(list_documents, text_splitter, embed_model, save_to_file=True, file_path='drive/MyDrive/ia024a/projeto/entrega3/retriever/', add_info=None):
    '''
    Splits and embeds a list of documents.
    '''

    # splits list of documents into chunks and adds the source txt filename as metadata
    split_documents = text_splitter.create_documents([doc.page_content for doc in list_documents], metadatas=[{'ref': doc.metadata['source'].split('/')[-1]} for doc in list_documents])

    # embeds and indexes each chunk using FAISS
    vector_store = FAISS.from_documents(split_documents, embed_model)

    if save_to_file:
        filename = file_path + 'faiss__' + add_info + '__' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        vector_store.save_local(filename)

    return vector_store

In [ ]:
# textbooks loaded as documents
docs = load_documents()

In [ ]:
# chunk embedding model
emb_model_name = 'sentence-transformers/all-MiniLM-L6-v2'

In [ ]:
embedding_model = HuggingFaceBgeEmbeddings(model_name=emb_model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Recursive character text splitter

Analysis of paragraph length shows that, with exception of one textbook (Surgery_Schwartz), every other book has an average length of up to 600 characters. We'll try a variation of this window size of 300. The overlap size will be fixed at 10% of the window size.

In [ ]:
for chunk_size, chunk_overlap in zip([300, 600, 900], [30, 60, 90]):

    print(chunk_size, chunk_overlap)

    # create text splitter with specified chunk and overlap sizes
    rec_text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # create a vector store from that splitter
    tmp_faiss = create_vector_store(docs, rec_text_splitter, embedding_model, add_info=f'{chunk_size}_{chunk_overlap}')

300 30
600 60
900 90


### ReAct agent

In [ ]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
# react prompt
react_prompt_template = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt = ChatPromptTemplate.from_template(react_prompt_template)

In [ ]:
llm = ChatOpenAI(
        model=model,
        temperature=model_temp,
    )

In [ ]:
retrievers = [
    '/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__300_30__20241105_235516',
    '/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__500_50__20241103_135850',
    '/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__600_60__20241106_000142',
    '/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__900_90__20241106_000721',
]

In [ ]:
for retriever in retrievers:

    print(retriever)

    # load saved faiss indexes
    faiss = FAISS.load_local(
        folder_path=retriever,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )

    # create a search tool from the faiss retriever
    tool_search_faiss = create_retriever_tool(
        retriever=faiss.as_retriever(search_kwargs={'k': 3}),
        name='search_docs',
        description='Searchs the query in documents and returns the top-3 most relevant ones.',
    )

    # create agent with tools
    agent = create_react_agent(
        llm=llm,
        tools=[tool_search_faiss],
        prompt=react_prompt
    )

    agent_executor = AgentExecutor(
        agent=agent,
        tools=[tool_search_faiss],
        verbose=False,
        handle_parsing_errors=True,
        return_intermediate_steps=True,
        max_iterations=5
    )

    tmp = retriever.split('/')[-1].split('__')[1]
    filename = f'/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_{tmp}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'

    # run agent
    with open(filename, 'w') as f:

        f.write(''.center(10, '-'))
        f.write('\n')
        f.write(f'Model: {model}\n')
        f.write(f'Temperature: {model_temp}\n')
        f.write(f'Prompt: {react_prompt}\n')
        f.write(''.center(10, '-'))
        f.write('\n[{')

        for n, question in enumerate(tqdm(sampled_data)):

            res = agent_executor.invoke({'question': question['question'], 'options': question['options']})
            f.write(f'"{n}": ')

            # serialize results as a string
            f.write(dumps(res))
            f.write(',')

        f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
        f.write('}]')

/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__300_30__20241105_235516


100%|██████████| 50/50 [09:24<00:00, 11.30s/it]


/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__500_50__20241103_135850


100%|██████████| 50/50 [05:43<00:00,  6.88s/it]


/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__600_60__20241106_000142


100%|██████████| 50/50 [07:53<00:00,  9.46s/it]


/content/drive/MyDrive/ia024a/projeto/entrega3/retriever/faiss__900_90__20241106_000721


100%|██████████| 50/50 [07:21<00:00,  8.82s/it]


#### Results with FAISS in 50 sample questions

In [ ]:
result_logs = [
    '/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_300_30_20241106_002253.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_500_50_20241106_003228.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_600_60_20241106_003821.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega3/results/answers_faiss_900_90_20241106_004629.txt'
]

In [ ]:
acc_df = pd.DataFrame(columns=['Chunk Size', 'Overlap Size', 'Accuracy'])

In [ ]:
for logfile in result_logs:

    chunk_size = logfile.split('/')[-1].split('_')[2]
    chunk_overlap = logfile.split('/')[-1].split('_')[3]

    print(chunk_size, chunk_overlap)

    with open(logfile) as f:

        # load log file with LLM results
        tmp = f.readlines()

        # recover results as a dictionary
        recovered_results = loads(tmp[-1])[0]

        limit_error = list()
        not_parsed = list()
        answers = list()

        # matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
        # e.g. A (answer text) => A
        # other cases will be ignored
        ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

        for n, (k, v) in enumerate(recovered_results.items()):

            # skip placeholder key
            if k == 'placeholder':
                continue

            # max_iteration set to 5
            if v['output'] == 'Agent stopped due to iteration limit or time limit.':
                answers.append(None)
                limit_error.append(k)

            elif len(v['output']) > 1:
                tmp_output = ans_ptn.findall(v['output'])
                if len(tmp_output) == 1:
                    answers.append(tmp_output[0])
                elif len(tmp_output) > 1:
                    answers.append(None)
                    not_parsed.append(k)
                    print(tmp_output)
                else:
                    answers.append(None)
                    not_parsed.append(k)
                    print(k, v['output'])

            else:
                if v['output'] in ['A', 'B', 'C', 'D']:
                    answers.append(v['output'])
                else:
                    answers.append(None)

    # unanswered questions or answers not properly parsed
    print('Unanswered questions or answers not properly parsed: ', len(limit_error))

    # parsing errors/other errors
    print('Parsing errors/other errors: ', len(not_parsed))

    # accuracy
    accuracy = sum([pred == gt for gt, pred in zip(sampled_data['answer_idx'], answers)]) / len(sampled_data)
    print('Accuracy: ', accuracy)
    print('\n')

    acc_df.loc[len(acc_df)] = [chunk_size, chunk_overlap, accuracy]

300 30
Unanswered questions or answers not properly parsed:  5
Parsing errors/other errors:  0
Accuracy:  0.66


500 50
Unanswered questions or answers not properly parsed:  4
Parsing errors/other errors:  0
Accuracy:  0.74


600 60
Unanswered questions or answers not properly parsed:  1
Parsing errors/other errors:  0
Accuracy:  0.72


900 90
Unanswered questions or answers not properly parsed:  3
Parsing errors/other errors:  0
Accuracy:  0.72




In [ ]:
acc_df

,Chunk Size,Overlap Size,Accuracy
0,300,30,0.66
1,500,50,0.74
2,600,60,0.72
3,900,90,0.72


### Retrieved context evaluation with RAGAS

In [ ]:
# result_dict[question]['intermediate_steps'] -> List[List[AgentAction(tool, tool_input, log), response: str]]
def format_context(answer):

    formatted_output = ''
    for i in answer['intermediate_steps']:
        formatted_output += i[1]

    return formatted_output

In [ ]:
class RelevantSentence(BaseModel):
    '''A sentence you believe is relevant to answer the provided question.
    '''
    sentence: str

class Context(BaseModel):
    '''The list of contexts you retrieved based on a question.
    '''
    contexts: List[RelevantSentence]

In [ ]:
ext_sentence_prompt = '''
Please extract relevant sentences from the provided context that can potentially help answer the following question.
If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase "Insufficient Information".
While extracting candidate sentences you’re not allowed to make any changes to sentences from given context.
Here is the context: {context}
And here is the question: {question}
'''.strip()

ext_sentence_prompt_template = ChatPromptTemplate.from_template(ext_sentence_prompt)

In [ ]:
chain_extraction = ext_sentence_prompt_template | llm.with_structured_output(Context)

In [ ]:
ctxt_rel_df = pd.DataFrame(columns=['Chunk Size', 'Overlap Size', 'Context relevancy'])

In [ ]:
for logfile in result_logs:

    chunk_size = logfile.split('/')[-1].split('_')[2]
    chunk_overlap = logfile.split('/')[-1].split('_')[3]

    print(chunk_size, chunk_overlap)

    # load log file with LLM results
    with open(logfile) as f:
        tmp = f.readlines()

    # recover results as a dictionary
    recovered_results = loads(tmp[-1])[0]

    # format recovered contexts as a list of strings
    react_logs = list()
    for i in recovered_results.values():
        if i == 'placeholder':
            continue
        react_logs.append(format_context(i))

    sampled_data_2 = sampled_data.add_column('answer_react_faiss', answers)
    sampled_data_2 = sampled_data_2.add_column('react_log', react_logs)

    relevant_contexts = list()
    context_rel = list()

    for question in tqdm(sampled_data_2):

        try:
            # select relevant sentences
            contexts = chain_extraction.invoke({'context': question['react_log'], 'question': question['question']})
            relevant_contexts.append(contexts)

            # fraction of relevant sentences in the context
            context_rel.append(len(contexts.contexts) / len(question['react_log'].split('.')))
        except:
            relevant_contexts.append(None)
            context_rel.append(np.nan)

    cont_rel_dataset = np.nanmean(context_rel)
    ctxt_rel_df.loc[len(ctxt_rel_df)] = [chunk_size, chunk_overlap, cont_rel_dataset]


300 30


100%|██████████| 50/50 [01:42<00:00,  2.05s/it]


500 50


100%|██████████| 50/50 [02:50<00:00,  3.40s/it]


600 60


100%|██████████| 50/50 [02:37<00:00,  3.15s/it]


900 90


100%|██████████| 50/50 [03:02<00:00,  3.65s/it]


In [ ]:
ctxt_rel_df

,Chunk Size,Overlap Size,Context relevancy
0,300,30,0.623789
1,500,50,0.416302
2,600,60,0.323267
3,900,90,0.330581
